In [23]:
# import packages
from collections import deque
import pandas as pd
import random
import copy

In [5]:
def find_free_partner(boys, girls, sort_boy_to_girl, sort_girl_to_boy):
    
    # current choice at hand
    current_boys = dict(zip(boys, [None]*len(boys)))
    current_girls = dict(zip(girls, [None]*len(girls)))
    # sumboys= sum(current_girls)
    # current_boys = {boys[0]:None, boys[1]:None, boys[2]:None, boys[3]:None}
    # current_girls = {girls[0]:None, girls[1]:None, girls[2]:None, boys[3]:None}
    count = len(boys)
 
 
    # array: boy's next choice (if rejected)
    next_select = dict(zip(boys, [None]*len(boys)))
    for i in range(count):
        temp = [girls[m-1] for m in sort_boy_to_girl[i]]
        next_select[boys[i]] = deque(temp)
 
 
    # dict: girl chooses boy
    sort_girl = dict(zip(girls, [None]*len(boys)))
    for i in range(count):
        # rank boys
        temp = [[boys[m-1], 4-ind] for ind, m in enumerate(sort_girl_to_boy[i])]
        
        name = []
        match = []
        for t in temp:
            name.append(t[0])
            match.append(t[1])
        
        sort_girl[girls[i]] = dict(zip(name, match))
 
 
    while None in current_boys.values():
        for i in range(count):
            bid = boys[i]
            if current_boys[bid]:
                # skip if boy is on the string
                continue
            else:
                # preferred girl
                select = next_select[bid][0]
                if current_girls[select] == None:
                    # match if both are single
                    current_boys[bid] = select
                    current_girls[select] = bid
                    next_select[bid].popleft()
                else:
                    # comparision
                    # if current > new then stay
                    if sort_girl[select][current_girls[select]] > sort_girl[select][bid]:
                        next_select[bid].popleft()
                    # else: opposite
                    # if new > current then change
                    # girl chooses new boy, current boy loses
                    # can't propose twice to a same girl
                    else:
                        current_boys[current_girls[select]] = None
                        current_boys[bid] = select
                        current_girls[select] = bid
                        next_select[bid].popleft()
    return current_boys

In [6]:
size=20
boys = list(range(1,size+1))
girls = list(range(1,size+1))
# rep: repeated time

In [7]:
# random preference matrix for boys
sort_boy_to_girl= [random.sample(range(1,size+1),size) for _ in range(1, size+1)]
#print(sort_boy_to_girl)
# preference of boys
dfboys = pd.DataFrame(pd.Series(sort_boy_to_girl),columns={'boy_preferece'})
dfboys.index=dfboys.index+1
dfboys=dfboys.reset_index().rename(columns = {'index':'boy'})

In [8]:
# random preference matrix for girls
sort_girl_to_boy= [random.sample(range(1,size+1),size) for _ in range(1, size+1)]
# print(sort_girl_to_boy)
dfgirls = pd.DataFrame(pd.Series(sort_girl_to_boy),columns={'girl_preference'})
dfgirls.index=dfgirls.index+1
dfgirls=dfgirls.reset_index().rename(columns = {'index':'girl'})

In [27]:
mpref=sort_boy_to_girl
wpref=sort_girl_to_boy

In [28]:
def gale_shapley(mpref, wpref, n, F):
    ### returns men's matchings or proposers' matchings i.e. men_matchings[i] is a partner of proposer i
    curr_match_count = 0
    last_rejected_by_rank_what = [0 for i in range(n)]
    men_matchings = [0 for i in range(n)]
    women_matchings = [0 for i in range(n)]

    # continue until all matches made
    while curr_match_count < n:
        for man in range(n):
            # if man is single
            if men_matchings[man] == 0:
                # get first non-rejecting woman from man's preference list
                woman = mpref[man][last_rejected_by_rank_what[man]] - 1
                # both man and woman are single
                if women_matchings[woman] == 0:
                    # create match (man, woman)
                    men_matchings[man] = woman + 1
                    women_matchings[woman] = man + 1
                    curr_match_count += 1

                # woman is with someone
                else:
                    # find ranks of curr_partner and man given woman's preference list
                    curr_partner = women_matchings[woman]
                    curr_partner_rank = wpref[woman].index(curr_partner)
                    man_rank = wpref[woman].index(man + 1)

                    # woman prefers curr_partner over man
                    if curr_partner_rank < man_rank:
                        last_rejected_by_rank_what[man] = mpref[man].index(woman + 1) + 1

                    # woman prefers man over curr_partner
                    else:
                        men_matchings[curr_partner - 1] = 0
                        last_rejected_by_rank_what[curr_partner - 1] = mpref[curr_partner - 1].index(woman + 1) + 1
                        men_matchings[man] = woman + 1
                        women_matchings[woman] = man + 1
    if F:
        men_matchings = women_matchings
    for x in range(n):
        men_matchings[x] -= 1
    return men_matchings

def get_reduced(mpref, wpref, matches, n):
    # matches is an output of GS, ie. a vector of proposers' partners
    mpref_red = copy.deepcopy(mpref)
    wpref_red =copy.deepcopy(wpref)
    for i in range(n):
        woman = i
        man = matches.index(woman)
        man_rank = wpref[woman].index(man)
        for i in range(man_rank + 1, len(wpref[woman])):
            successor = wpref[woman][i]
            wpref_red[woman].remove(successor)
            # print("removing {0} from {1} list".format(woman, successor))
            mpref_red[successor].remove(woman)
    return mpref_red, wpref_red

In [36]:
mpref

[[10, 18, 5, 16, 17, 11, 20, 9, 6, 14, 1, 7, 8, 12, 4, 2, 13, 15, 19, 3],
 [1, 13, 4, 12, 6, 15, 2, 3, 20, 9, 17, 11, 8, 18, 7, 19, 16, 10, 5, 14],
 [11, 18, 1, 2, 12, 5, 16, 15, 14, 13, 6, 10, 9, 7, 19, 3, 4, 20, 17, 8],
 [6, 17, 9, 3, 19, 16, 5, 12, 7, 10, 20, 11, 15, 8, 18, 14, 2, 13, 1, 4],
 [11, 16, 1, 3, 9, 15, 10, 19, 2, 13, 4, 18, 12, 17, 7, 8, 20, 6, 5, 14],
 [16, 7, 20, 18, 17, 8, 2, 11, 3, 12, 19, 9, 10, 14, 13, 15, 4, 1, 5, 6],
 [15, 6, 4, 8, 11, 17, 2, 5, 19, 14, 9, 16, 1, 12, 13, 10, 3, 20, 18, 7],
 [20, 14, 11, 10, 18, 6, 3, 9, 19, 5, 7, 15, 1, 8, 17, 4, 13, 12, 16, 2],
 [19, 16, 11, 7, 6, 15, 18, 13, 9, 12, 8, 2, 1, 14, 17, 3, 20, 10, 4, 5],
 [11, 4, 1, 6, 18, 2, 8, 14, 10, 17, 9, 3, 5, 7, 12, 20, 15, 19, 16, 13],
 [14, 7, 20, 3, 4, 11, 9, 13, 6, 8, 17, 12, 16, 19, 1, 15, 18, 2, 5, 10],
 [17, 11, 10, 18, 20, 14, 12, 9, 19, 8, 2, 4, 3, 6, 15, 13, 5, 16, 1, 7],
 [19, 13, 9, 20, 12, 2, 8, 6, 15, 5, 16, 11, 14, 7, 3, 1, 17, 10, 18, 4],
 [18, 5, 8, 11, 15, 1, 17, 16, 20, 19,

In [37]:
wpref

[[10, 16, 17, 5, 15, 19, 18, 20, 9, 12, 14, 6, 4, 2, 3, 1, 11, 13, 8, 7],
 [20, 16, 5, 2, 7, 18, 1, 13, 8, 3, 19, 4, 11, 9, 17, 12, 6, 14, 10, 15],
 [19, 13, 12, 2, 1, 10, 18, 11, 5, 7, 15, 16, 8, 3, 14, 6, 4, 9, 17, 20],
 [18, 14, 20, 7, 11, 9, 15, 2, 1, 10, 8, 16, 5, 12, 17, 3, 4, 13, 19, 6],
 [12, 11, 8, 7, 2, 19, 10, 16, 14, 5, 1, 6, 18, 15, 4, 20, 9, 3, 17, 13],
 [13, 2, 4, 5, 10, 16, 8, 15, 20, 17, 11, 18, 7, 19, 9, 14, 12, 6, 3, 1],
 [9, 2, 3, 7, 16, 18, 4, 8, 17, 14, 10, 1, 11, 20, 13, 19, 6, 5, 12, 15],
 [11, 16, 7, 5, 3, 8, 12, 4, 10, 13, 6, 20, 1, 19, 17, 18, 14, 2, 15, 9],
 [7, 10, 5, 19, 6, 4, 18, 17, 9, 3, 20, 2, 13, 1, 8, 11, 16, 12, 14, 15],
 [16, 11, 17, 15, 3, 6, 2, 10, 1, 20, 8, 9, 5, 18, 7, 14, 12, 4, 19, 13],
 [17, 19, 6, 3, 10, 9, 16, 14, 4, 11, 2, 12, 5, 1, 15, 8, 13, 20, 7, 18],
 [20, 8, 9, 11, 7, 19, 17, 16, 18, 3, 2, 1, 4, 12, 10, 6, 5, 13, 14, 15],
 [18, 10, 16, 13, 19, 20, 4, 5, 1, 7, 12, 11, 9, 14, 15, 8, 3, 17, 6, 2],
 [11, 3, 16, 8, 12, 1, 17, 2, 9, 15, 1

In [40]:
df = pd.DataFrame(pd.Series(gale_shapley(mpref, wpref, 20, 1)))
df

,0
0,9
1,19
2,4
3,1
4,14
5,3
6,8
7,15
8,7
9,0


In [41]:
def get_reduced(mpref, wpref, matches, n):
    # matches is an output of GS, ie. a vector of proposers' partners
    mpref_red = copy.deepcopy(mpref)
    wpref_red =copy.deepcopy(wpref)
    for i in range(n):
        woman = i
        man = matches.index(woman)
        man_rank = wpref[woman].index(man)
        for i in range(man_rank + 1, len(wpref[woman])):
            successor = wpref[woman][i]
            wpref_red[woman].remove(successor)
            # print("removing {0} from {1} list".format(woman, successor))
            mpref_red[successor].remove(woman)
    return mpref_red, wpref_red

In [45]:
def find_all_stable(malechoice, femalechoice, n):
    # Output list with all matchings
    res = []
    # Initialization
    unchanged = [True] * n
    marriage = [0] * n
    malecounter = [0] * n
    # Female choice transformation

    fc = np.zeros((n, n), dtype=int)
    for i in range(1, n):
        for j in range(0, n - 1):
            fc[i][femalechoice[i][j]] = j
        fc[i][0] = n
        malecounter[i] = 0
        marriage[i] = 0

    def get_marriage(marriage):
        malematches = [0] * (n - 1)
        for j in range(1, n):
            malematches[marriage[j] - 1] = j
        return(malematches)

    def print_marriage(marriage):
        malematches = [0] * (n - 1)
        for j in range(1, n):
            malematches[marriage[j] - 1] = j
        print(*malematches)

    def proposal(i, malec, marriage, c):
        if i < 0:
            success = True
        elif (i == 0) or malec[i] == (n - 1) or not (unchanged[i]):  # n not (n+1), as we have n equals 5
            success = False
        else:
            c += 1
            j = malec[i]
            malec[i] = j + 1
            success = refusal(i, malechoice[i][j], malec, marriage, c)
        return success

    def refusal(i, j, malec, marriage, c):
        if fc[j][abs(marriage[j])] > fc[j][i]:
            k = marriage[j]
            marriage[j] = i
            success = proposal(k, malec, marriage, c)
        else:
            success = proposal(i, malec, marriage, c)
        return success

    def breakmariage(malecounter, marriage, i, n, count):
        malecounter_ori = malecounter.copy()
        marriage_ori = marriage.copy()
        marriage[malechoice[i][malecounter[i] - 1]] = -i
        success = proposal(i, malecounter, marriage, count)

        if not (success):
            unchanged[i] = False
            malecounter_ref = malecounter_ori.copy()
            marriage_ref = marriage_ori.copy()
            malecounter = malecounter_ref
            marriage = marriage_ref
            return malecounter, marriage

        #print_marriage(marriage)  # marriage[i] means the woman who is married to i man
        new_marriage = get_marriage(marriage)  # convert into female solution
        res.append(new_marriage)
        # establish the relationships between the original and the obtained stable matching

        for j in range(i, n):
            # if i==2 and j==2:
            #     print("This is it")
            malecounter, marriage = breakmariage(malecounter, marriage, j, n, count)

        for j in range((i + 1), n):
            unchanged[j] = True
        # return malecounter_ori, abs(marriage_ori)
        return malecounter_ori, marriage_ori

    count = 0
    for i in range(1, n):
        proposal(i, malecounter, marriage, count)
    #print_marriage(marriage)

    new_marriage = get_marriage(marriage)  # convert into female solution
    res.append(new_marriage)
    # establish the relationships between the original and the obtained stable matching

    M_optimal_marriage = marriage.copy()
    M_optimal_counter = malecounter.copy()
    for i in range(1, n - 1):
        breakmariage(M_optimal_counter.copy(), M_optimal_marriage.copy(), i, n, count)
    return res

In [9]:
# initialization
# size: population, boy = girl here 
size=20
boys = list(range(1,size+1))
girls = list(range(1,size+1))
# rep: repeated time
rep=1000

# create empty variables
# mr_girl_sum: matching rank of GIRLS, accumulated of each time repeated 
# mr_boy_sum: matching rank of BOYS, accumulated of each time repeated 
mr_girl_sum=0
mr_boy_sum=0

# amr_girl: average matching rank of GIRLS (mr_girl_sum/rep)
# amr_boy: average matching rank of GIRLS (mr_boy_sum/rep)
amr_girl=0
amr_boy=0
# repeat
for i in range(1,rep):
    # set random preference matrices
    # preference of boys
    sort_boy_to_girl= [random.sample(range(1,size+1),size) for _ in range(1, size+1)]
    # preference of girls
    sort_girl_to_boy= [random.sample(range(1,size+1),size) for _ in range(1, size+1)]
    # run the function
    girl_match=pd.Series(find_free_partner(boys, girls, sort_boy_to_girl, sort_girl_to_boy))
    # one time
    for i in range(1,size):
        # the rank of matched girl for the boy
        mr_boy=sort_boy_to_girl[i-1].index(girl_match[i])+1
        # the rank of matched boy for the girl
        mr_girl=sort_girl_to_boy[i-1].index(i)
        # sum up ranks of matched girls for all boys
        mr_boy_sum += mr_boy
        # sum up ranks of matched boys for all girls
        mr_girl_sum += mr_girl
        i+=1
    # average rank of matched girl for boys, in rep
    amr_boy+=mr_boy_sum/size/rep
    # average rank of matched boy for girls, in rep
    amr_girl+=mr_girl_sum/size/rep
    i+=1
print("amr_boy=",amr_boy/rep)
print("amr_girl=",amr_girl/rep)
print("amr_total=",amr_boy/rep/2+amr_girl/rep/2)
print("fe=",abs(amr_boy/rep/2-amr_girl/rep/2))

amr_boy= 1.4765240999999996
amr_girl= 4.499852850000002
amr_total= 2.9881884750000007
fe= 1.511664375000001


In [10]:
# random preference matrix
sort_girl_to_boy= [random.sample(range(1,size+1),size) for _ in range(1, size+1)]
# print(sort_girl_to_boy)
dfgirls = pd.DataFrame(pd.Series(sort_girl_to_boy),columns={'girl_preference'})
dfgirls.index=dfgirls.index+1
dfgirls=dfgirls.reset_index().rename(columns = {'index':'girl'})

In [11]:
# create a dataframe
df = pd.DataFrame(pd.Series(find_free_partner(boys, girls, sort_boy_to_girl, sort_girl_to_boy)), columns=['girl_match'])
df=df.reset_index().rename(columns = {'index':'boy_match'})

In [12]:
# calculate amr for boys
girl_match=pd.Series(find_free_partner(boys, girls, sort_boy_to_girl, sort_girl_to_boy))
mr_boy_sum=0
for i in range(1,size):
    mr_boy=sort_boy_to_girl[i-1].index(girl_match[i])
   # print("mr_boy=",mr_boy)
    mr_boy_sum+=mr_boy+1
    # print("mr_boy_sum=",mr_boy_sum)
    i+=1
# print(mr_boy_sum)
amr_boy=mr_boy_sum/size
print("amr_boy=",amr_boy)

# calculate amr for girls
mr_girl_sum=0
for i in range(1,size):
    mr_girl=sort_girl_to_boy[i-1].index(i)
    mr_girl_sum+=mr_girl
    i+=1
amr_girl=mr_girl_sum/size
print("amr_girl=",amr_girl)

amr_boy= 2.65
amr_girl= 10.2


In [13]:
# crossover probability
# cycle crossover operation
prob=0.6
size=20

crossover= random.sample(range(1,size+1),12)
# crossover

In [14]:
df2=df[df["boy_match"].isin(crossover)]
# turn df2['boy_match'] into list of boys divorced
boy_divorce=df2['boy_match'].values.tolist()
# full preference of boys divorced
sort_boy_to_girl_divorce_full=[n for i, n in enumerate(sort_boy_to_girl) if i+1 in crossover]

#print(boy_divorce)
#print(sort_boy_to_girl_divorce_full)
# turn girl_match into list of girls divorced
girl_divorce=df2['girl_match'].values.tolist()
#print(girl_divorce)
# full preference of girls divorced
sort_girl_to_boy_divorce_full=[n for i, n in enumerate(sort_girl_to_boy) if i in df2['girl_match']]
#print(sort_girl_to_boy_divorce_full)


In [15]:
sort_boy_to_girl_divorce=[]
for j in range(12):
    lis = [i+1 for i, n in enumerate(sort_boy_to_girl_divorce_full[j]) if n in girl_divorce]
    sort_boy_to_girl_divorce.append(lis)
#sort_boy_to_girl_divorce

In [16]:
sort_girl_to_boy_divorce=[]
for j in range(12):
    lis = [i+1 for i, n in enumerate(sort_girl_to_boy_divorce_full[j]) if n in girl_divorce]
    sort_girl_to_boy_divorce.append(lis)
#sort_girl_to_boy_divorce

In [17]:
sort_boy_to_girl_divorce_2=[]
for j in range(12):
    lis = [i+1 for i, n in enumerate(sort_boy_to_girl_divorce[j]) if n in boy_divorce]
    sort_boy_to_girl_divorce_2.append(lis)
sort_boy_to_girl_divorce_2

[[1, 2, 4, 7, 11, 12],
 [1, 2, 3, 6, 7, 8, 12],
 [1, 2, 5, 6, 7, 9, 10],
 [1, 2, 3, 4, 7, 8, 11, 12],
 [3, 4, 7, 10, 11, 12],
 [1, 2, 5, 6, 7, 8, 10, 11, 12],
 [1, 2, 5, 7, 10, 11, 12],
 [1, 2, 3, 6, 8, 10, 11, 12],
 [1, 4, 5, 7, 11, 12],
 [1, 2, 3, 7, 10, 12],
 [1, 5, 11, 12],
 [1, 4, 5, 6, 10, 11, 12]]

In [18]:
sort_girl_to_boy_divorce_2=[]
for j in range(12):
    lis = [i+1 for i, n in enumerate(sort_girl_to_boy_divorce[j]) if n in girl_divorce]
    sort_girl_to_boy_divorce_2.append(lis)
sort_girl_to_boy_divorce_2

[[1, 2, 5, 6, 7, 8, 10, 11],
 [1, 2, 5, 8, 10, 11],
 [1, 2, 4, 7, 8, 10, 12],
 [1, 2, 3, 6, 8, 9, 10, 11],
 [1, 3, 5, 6, 7, 9, 10, 11],
 [1, 2, 3, 5, 6, 7, 8, 11],
 [1, 2, 3, 5, 6, 8, 10, 11],
 [1, 2, 4, 6, 7, 8, 10, 11],
 [5, 6, 9, 10],
 [1, 2, 7, 8, 10, 11],
 [1, 3, 5, 6, 8, 9, 11],
 [1, 3, 5, 6, 7, 8, 10, 11]]

In [19]:
def find_free_partner_divorce(boys_divorce, girl_divorce, sort_boy_to_girl_divorce, sort_girl_to_boy_divorce):
    
    # current choice at hand
    current_boys_divorce = dict(zip(boys_divorce, [None]*len(boys_divorce)))
    current_girls_divorce = dict(zip(girl_divorce, [None]*len(girl_divorce)))
    sumboys= sum(current_girls_divorce)
    current_boys_divorce = {boys[0]:None, boys[1]:None, boys[2]:None, boys[3]:None}
    current_girls_divorce = {girls[0]:None, girls[1]:None, girls[2]:None, boys[3]:None}
    count = len(boys_divorce)
 
 
    # array: boy's next choice (if rejected)
    next_select = dict(zip(boys_divorce, [None]*len(boys_divorce)))
    for i in range(count):
        temp = [girl_divorce[m-1] for m in sort_boy_to_girl_divorce[i]]
        next_select[boys_divorce[i]] = deque(temp)
 
 
    # dict: girl chooses boy
    sort_girl = dict(zip(girl_divorce, [None]*len(boys_divorce)))
    for i in range(count):
        # rank boys
        temp = [[boys_divorce[m-1], 4-ind] for ind, m in enumerate(sort_girl_to_boy_divorce[i])]
        
        name = []
        match = []
        for t in temp:
            name.append(t[0])
            match.append(t[1])
        
        sort_girl[girl_divorce[i]] = dict(zip(name, match))
 
 
    while None in current_boys_divorce.values():
        for i in range(count):
            bid = boys_divorce[i]
            if current_boys_divorce[bid]:
                # skip if boy is on the string
                continue
            else:
                # preferred girl
                select = next_select[bid][0]
                if current_girls_divorce[select] == None:
                    # match if both are single
                    current_boys_divorce[bid] = select
                    current_girls_divorce[select] = bid
                    next_select[bid].popleft()
                else:
                    # comparision
                    # if current > new then stay
                    if sort_girl[select][current_girls_divorce[select]] > sort_girl[select][bid]:
                        next_select[bid].popleft()
                    # else: opposite
                    # if new > current then change
                    # girl chooses new boy, current boy loses
                    # can't propose twice to a same girl
                    else:
                        current_boys_divorce[current_girls_divorce[select]] = None
                        current_boys_divorce[bid] = select
                        current_girls_divorce[select] = bid
                        next_select[bid].popleft()
    return current_boys_divorce

In [20]:
df = pd.DataFrame(pd.Series(find_free_partner_divorce(boy_divorce, girl_divorce, sort_boy_to_girl_divorce_2, sort_girl_to_boy_divorce_2)), columns=['girl_match'])


KeyError: 10